In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams['axes.facecolor'] = '#323A48'
plt.rcParams['axes.edgecolor'] = '#92A2BD'
plt.rcParams['figure.facecolor'] = '#323A48'
plt.rcParams['text.color'] = '#DBE1EA'
plt.rcParams['xtick.color'] = '#DBE1EA'
plt.rcParams['ytick.color'] = '#DBE1EA'
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['grid.color'] = '#3F495A'

from cycler import cycler
plt.rcParams['axes.prop_cycle'] = cycler(color=['c', 'goldenrod', 'mediumseagreen', 'blueviolet', 'indianred', 'mediumpurple', 'cornflowerblue',
                                               'darkorchid', 'gold', 'olivedrab'])
import time

## 24 hour format ##
def print_t(str_):
  print( "[" + time.strftime("%Y-%m-%d %H:%M:%S") + "] " + str(str_))
  
def timestamp():
  return time.strftime("%y%m%dd%Hh%Mm%S")

In [4]:
import socket
import json

class CCGame:
  def __init__(self, log_ticks=False):
    self.log_ticks = log_ticks
    self.tickLogs = []
    self.tcp_ip = '31.46.64.35'
    self.tcp_port = 12323
    self.team_token = '1iVXOVZK7ldH5Kr6qYCEkZE6xpR0SXZJkyfQayrKfJ2e9S8xdeTjsV9oohjePSsUXFOcDnevsu918'
    self.buffersize = 1024
    self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    self.valid_commands = np.array(["NO_OP", "ACCELERATION", "DECELERATION", "CAR_INDEX_LEFT", "CAR_INDEX_RIGHT", "CLEAR", "FULL_THROTTLE", "EMERGENCY_BRAKE", "GO_LEFT", "GO_RIGHT"])
    
    # Build map 
    mapstr = "GPSSPGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGPSSPGPPSSPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPSSPPSSSSSSSSSSSSSSSSSSSSSSSSSSSSZSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSZSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPSSPPPZZPPPPPPPPPPPPPPPPPPPPSSPPPPPPPPPPPPPPPPPPPPZZPPPSSPPGPSSPGPSSPGBBGBBGGBBGGBBGBBGPSSPGBBGBBGGBBGGBBGBBGPSSPBPSSPGGPSSPBPSSPPPPPPPPPPPPPPPPPPPPSSPPPPPPPPPPPPPPPPPPPPSSPBPSSPGGPSSPBPSSSSSSSSZSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPGPSSPGGPSSPBPSSSSSSSSZSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPBPSSPGGPSSPBPSSPPPPPPPSSPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPSSPBPSSPGGPSSPGPSSSPPGBBPSSPGGGGGBBBBBGPPPPPPPPGGBBBBBBGBBPSSSPGPSSPGGPSSPBPSSSSPPGBPSSPGTGPPPPPPPPPSSSSSSPPPBBBBBBGBBPSGSPBPSSPGGPSSPBPPSSSSPPGPSSPGGGPSSSSSSSSSSSSSSSSPPPGGGPPPPPSGSPBPSSPGGPSSPGGPPSSSSPPPSSPPBBPSSSSSSSSSSPPSSSSSSPPPGPSSSSSGSPGPSSPGGPSSPGGGPPSSSSPSSSSPPPPSSPSSPPSSPPPPPSSSSSSPPPSSSSSSSPGPSSPGGPSSPGGGGPPSSSSSBBSSSSSSSSSSPPSSSSSSPPPSSSSSSZSSPPPPPPGPSSPGGPSSPGGGGGPPSSSSBBSSSSSSSSSSPPSPPPPSSSPPPSSSSZSSPPSSPGGPSSPGGPSSPGGGGGGPPPPSSSSPPPPPPPPPPPSPPSSSSSSPPPPPPPSSPPSSPTTPSSPGGPSSPGGGGGGGGGPPSSPPSSSSSSSSGPSSSSPPSSSSPPGTGPSSSSSSPTGPSSPGGPSSPGGGGGGBBBGPSSSSSSSSSSSSSPPPPPPPPSSSSPPGPPSSSSSSPTTPSSPGGPSSPGGGGGGBBBGPSSSSSSPPPPSSSSSSSSSSPPSSSSPPPSSSPPPPPTTPSSPGGPSSPGGGGGGBBBGPSSPPPPPGGPPSSSSSSSSSSPPSSSSPSSSSPBBBBTGPSSPGGPSSPGPPPPPPPPPPSSPGGGGGGGPPPPPPPPSSSSPPSSSSSSSPPBBBBBGPSSPGGPSSPGPSSSSSSSSSSSPGGGGGGGGGGGBBGPPSSSSPPSSSSSPPGGGGGGGPSSPGGPSSPGPSSSSSSSSSSSPGGGGBGGGGGGBBGGPPSSSSPPPPSSSPPPPPPGTPSSPGGPSSPGPSSPPPPPPPSSPGGGGGGGGGGGGGGGGPPSSSSSSPSSSSSSSSPGTPSSPGGPSSPGPSSPBGBBBPSSPGGGGGGGGGGPPPPGGGPPSSSSSPPSSSSSSSPGTPSSPGGPSSPGPSSPBGBBBPSSPGGGGGGGGGPPSSPPGGGPPPPPPPPPPPPPZZPPGPSSPGGPZZPPPZZPPPPPPPSSPPPPPPPPPPPSSSSPPPPPPPPPPPPPPPPSSSSPPPZZPGGPSSSSSSSSSSSSSSSSSSSSSSSSSSSSBBSSSSSSSSSSSSSSSSSSBBSSSSSSPGGPSSSSSSSSSSSSSSSSSSSSSSSSSSSSBBSSSSSSSSSSSSSSSSSSBBSSSSSSPGGPZZPPPPPPPPPPPPPPPPSSPPPPPPPSSSSPPPPPPPSSPPPPPPPSSSSPPPZZPGGPSSPGPSSSSPPBBBBPPSSSPGGPPPPPSSPPPPPGGPSSSPPGBBPPSSPPGPSSPGGPSSPBPSSSSSPBBBBPSSSSPBBPSSSSSSSSSSPGGPSSSSPGBBGPSSPGBPSSPGGPSSPBPPPSSSPGGGGPSSSPPGGPSSSSSSSSSSPGGPPSSSPGGGGPSSPGBPSSPGGPSSPGGGPPSSPGGGGPSSPPGGGPSSPPPPPPSSPGGGPPSSPGGGGPSSPGBPSSPGGPSSPGGBBPSSPGGGGPSSPGGGBPSSPGGGGPSSPGGGGPSSPGGGGPSSPGBPSSPGGPSSPGGBBPSSPGGGGPSSPGBBBPSSPGGGGPSSPGGGGPSSPGGGGPSSPGBPSSPGGPSSPGPPPPSSPPPPPPZZPPPPPPZZPPPPPPSSPPPPPPZZPPPPPPZZPPGPSSPGGPSSPBPSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSZSSZSSSSSSSSPBPSSPGGPSSPBPSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSZSSZSSSSSSSSPBPSSPGGPSSPGPSSPPPPPSSPPPPPPPPPPPPZZPPPPPZZPPPPPSSPPPPPSSPPPBPSSPGGPSSPBPSSPGGGPSSPGGGGGGGBBBPSSPGGGPSSPBBGPSSPGBBPSSPBGBPSSPGGPSSPBPSSPGGGPSSPGGGGGGGBBBPSSPGGGPSSPBBGPSSPGBBPSSPBGBPSSPGGPSSPGPSSPPPPPZZPPPPPPPPPPPPSSPPPPPSSPPPPPSSPPPPPSSPPGGPSSPGGPSSPGPSSSSSSZSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPGBPSSPGGPSSPGPSSSSSSZSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPGBPSSPGGPSSPGPPSSPPPPZZPPPPSSPPPPZZPPPPSSPPPPPPPPPPSSPPPPZZPGBPSSPGGPSSPGGPSSPBBPSSPBBPSSPGBPSSPBGPSSPGGGGGGGGPSSPGGPSSPGBPSSPGGPSSPGGPSSPBBPSSPBBPSSPBBPSSPBGPSSPGGGGGGGGPSSPGGPSSPGBPSSPGGPSSPGGPSSPPPPSSPPPPZZPPPPSSPPPPZZPPPPPPPPPPZZPPPPSSPGGPSSPGGPSSPGGPSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPGGPSSPGGPSSPGGPSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPGGGGPSSPGGPSSPGGPPPPPPPPPPPPPPPPPPPPPPSSPPPPPPPPPPPPPPPPPPPPGGGGPSSPGGPSSPGGGGGGGGGGGGGBBBBGGBBBBPSSPBBBBGGBBBBGGGGGGGGGGGGGPSSPGPPSSPPPPPPPPPPPPPPPPPPPPPPPPPSSPPPPPPPPPPPPPPPPPPPPPPPPPSSPPSSSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSZSSSSSSSSZSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSZSSSSPPSSPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPSSPPGPSSPGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGPSSPG"
    mapstrToInt = {
      'S': 0, # Aszfaltozott út
      'Z': 1, # Zebra
      'P': 2, # Járda
      'G': 3, # fű, zöld terület
      'B': 4, # épület
      'T': 5  # fa
    }
    self.mapMatrix = np.array([mapstrToInt[x] for x in list(mapstr)]).reshape(60,60)
    
    self.started = False
    
    self.tick_data_prev = None # Previous tick data
    self.tick_data = None # Current tick data 
    
    self.game_id = None
    self.ticknum = None
    self.car_id = None
    self.myCar_prev = None # Previous car object
    self.myCar = None # Current car object
    self.crashed = False
    return
    
  def get_response(self):
    if not self.started:
      raise RuntimeError("Game not started yet.")
    
    # Update tick_data and tick_data_prev
    if self.tick_data != None:
      self.tick_data_prev = self.tick_data
    self.tick_data = json.loads(self.socket.recv(self.buffersize).decode())
    if self.log_ticks==True:
      self.tickLogs.append(self.tick_data)
        
    if len(self.tick_data['messages']) > 0:
      if 'END REASON: CRASHED' in self.tick_data['messages']:
        self.crashed = True
        self.close()
        print_t('CAR CRASHED.')
      else:
        raise ValueError("Messages array is not empty:\n {}".format(self.tick_data['messages']))
    
    
    # Update myCar and myCar_prev
    if self.myCar != None:
      self.myCar_prev = self.myCar
      
    if len(self.tick_data['cars'])>0 and self.car_id!=None:
      self.myCar = [c for c in self.tick_data['cars'] if c['id']==self.car_id][0]
      
    self.game_id = self.tick_data['request_id']['game_id']
    self.ticknum = self.tick_data['request_id']['tick']
    self.car_id = self.tick_data['request_id']['car_id']
    return
    
    
#     if self.myCar != None:
#       self.last_life = self.myCar['life']
#       self.last_transported = self.myCar['transported']
#       if 'passenger_id' in self.myCar:
#         self.last_passenger_id=self.myCar['passenger_id']
#     return
  
  def start(self):
    if self.started == True:
      raise RuntimeError("Game already started.")
      
    self.socket.connect((self.tcp_ip, self.tcp_port))
    self.started = True
    self.tickLogs = []
    
    # Send first messsage
    first_message = {
      "token": self.team_token
    }
    first_message = json.dumps(first_message).encode()
    
    self.socket.send(first_message)
    self.get_response()
    return
    
  def close(self):
    if self.started == False:
      print_t("Game already closed.")
    self.socket.close()
    self.started = False
    if self.log_ticks==True:
      pass # Save tick logs to a file
    return
  
  def draw_map(self):
    plt.figure(figsize=(8,8), dpi=112)
    plt.imshow(g.mapMatrix)
    return
  
  def send_random_command(self):
    if self.started == False:
      raise RuntimeError("Game is stopped.")
    msg = {"response_id":
           {"game_id": self.game_id, "tick": self.ticknum, "car_id": self.car_id},
           "command": np.random.choice(self.valid_commands)
          }
    self.socket.send(json.dumps(msg).encode())
    self.get_response()
    return
    
  def send_command(self, command):
    if self.started == False:
      raise RuntimeError("Game is stopped.")
    msg = {"response_id":
           {"game_id": self.game_id, "tick": self.ticknum, "car_id": self.car_id},
           "command": command
          }
    self.socket.send(json.dumps(msg).encode())
    self.get_response()
    return

In [5]:
"""
  Implement a class that rewards the DQN agent
"""
class Rewarder:
  def __init__(self):
    pass
  
  def calculate_reward(self, game):
    rew = 0
    if game.myCar != None and game.myCar_prev != None:
      lifeDiff = game.myCar_prev['life'] - game.myCar['life']
      rew = rew-lifeDiff
      
      transportedDiff = game.myCar['transported'] - game.myCar_prev['transported'] 
      rew = rew + 200*transportedDiff
      
      if ('passenger_id' in game.myCar) and not ('passenger_id' in game.myCar_prev):
        rew = rew + 100
    return rew

In [6]:
"""
  Implement a class that can generate a state-space for the DQN based on tick data from server
"""
class StateSpace:
  def __init__(self):
    self.viewArea = [
    "         ",
    "   XXX   ",
    "   XXX   ",
    "   XXX   ",
    "   XXX   ",
    "  XXXXX  ",
    " XXXXXXX ",
    " XXXXXXX ",
    " XXXCXXX ",
    "  XXXXX  ",
    "   XXX   ",
    "    X    ",
    "         "]
    
    """ THIS FUNCTION IS TESTED """
    def _viewCoordsFromViewArea():
      carCoords = None
      seenCoords = []
      for y in range(len(self.viewArea)):
        line = list(self.viewArea[y])
        for x in range(len(line)):
          if line[x]=='C':
            carCoords = np.array([x,y])
          if line[x]=='X':
            seenCoords.append([x,y])
      return np.array([[c[0] - carCoords[0], c[1]-carCoords[1]] for c in seenCoords])
    
    self.viewAreaCoords = _viewCoordsFromViewArea()
    self.directionCodes = {
      # 0 is for no car
      'UP': 1,
      'DOWN': 2,
      'LEFT': 3,
      'RIGHT': 4
    }
    self.myCar = None
    pass
  
  """
    Get the coordinates of seen points relative to the car coordinates
    THIS FUNCTION IS TESTED
  """
  def relativeViewCoordsForDirection(self, d):
    def rmatrix(phi):
      return np.array([[np.cos(phi), -np.sin(phi)], [np.sin(phi), np.cos(phi)]])
    
    def dotnround(M, x):
      return np.round(M.dot(x)).astype(int)
    
    
    if d == 'UP':
      return self.viewAreaCoords
    elif d == 'DOWN':
      rotated = []
      R = rmatrix(np.pi)
      for c in self.viewAreaCoords:
        rotated.append(dotnround(R, c))
      return np.array(rotated)
    elif d == 'LEFT':
      rotated = []
      R = rmatrix(-np.pi/2.0)
      for c in self.viewAreaCoords:
        rotated.append(dotnround(R, c))
      return np.array(rotated)
    elif d == 'RIGHT':
      rotated = []
      R = rmatrix(np.pi/2.0)
      for c in self.viewAreaCoords:
        rotated.append(dotnround(R, c))
      return np.array(rotated)
    else: 
      raise ValueError("Invalid direction: "+d)
  
  """
    Build the state of the system, based on the current tick stored in game
  """
  def stateFromTick(self, game):
    if len(game.tick_data['cars'])>0:
      self.myCar = [c for c in game.tick_data['cars'] if c['id']==game.car_id][0]
    else:
      raise RuntimeError("Could not find my car.")
    
    print_t("My car is {}".format(self.myCar))
    
    # This is the state of the system that should be returned
    self.state = []
    
    # Check if the point is off the map
    def isOffMap(point):
      if point[0]<0 or point[0]>=60 or point[1]<0 or point[1]>=60:
        return True
      return False
    
    # Check if a car is on the point
    def carOnPoint(point):
      if len(game.tick_data['cars'])>=2:
        for car in game.tick_data['cars']:
          if car['pos']['x'] == point[0] and car['pos']['y'] == point[1]:
            return {"status": True, "car": car}
        return {"status": False, "car": None}
      return {"status": False, "car": None}
    
    # Check if a pedestrian is on the point
    def pedestrianOnPoint(point):
      if len(game.tick_data['pedestrians'])>=2:
        for ped in game.tick_data['pedestrians']:
          if ped['pos']['x'] == point[0] and ped['pos']['y'] == point[1]:
            return {"status": True, "ped": ped}
        return {"status": False, "ped": None}
      return {"status": False, "ped": None}
    
    # Check if a passenger is on the point
    def passengerOnPoint(point):
      if len(game.tick_data['passengers'])>=2:
        for pas in game.tick_data['passengers']:
          if pas['pos']['x'] == point[0] and pas['pos']['y'] == point[1]:
            return {"status": True, "pas": pas}
        return {"status": False, "pas": None}
      return {"status": False, "pas": None}
    
    # Iterate over all points seen by car
    relSeenCoords = self.relativeViewCoordsForDirection(self.myCar['direction'])
    for k in range(len(relSeenCoords)):
      mapPoint = [relSeenCoords[k][0]+self.myCar['pos']['x'],
                  relSeenCoords[k][1]+self.myCar['pos']['y']]
      
      if isOffMap(mapPoint):
        n_feat=11 # number of features in each point is 11
        self.state.append([-1 for k in range(n_feat)])
        continue
        
      car = carOnPoint(mapPoint)
      carDir = 0 if car['status']==False else self.directionCodes[car['car']['direction']]
      carSpeed = -1 if car['status']==False else car['car']['speed']
      
      ped = pedestrianOnPoint(mapPoint)
      pedDir = 0 if ped['status']==False else self.directionCodes[ped['ped']['direction']]
      pedSpeed = -1 if ped['status']==False else ped['ped']['speed']
      
      pas = passengerOnPoint(mapPoint)
      pasId = -1 if pas['status']==False else pas['pas']['id']
      pasDestX = -1 if pas['status']==False else pas['pas']['dest_pos']['x']
      pasDestY = -1 if pas['status']==False else pas['pas']['dest_pos']['y']
      pasIsInMyCar = -1 if pas['status']==False else int(self.myCar['id']==pas['pas']['car_id'])
      
      stateOfPoint = [
        game.mapMatrix[mapPoint[0]][mapPoint[1]], # What is the color of this point
        int(car['status']), # Is there any car on this point?
        carDir,
        carSpeed,
        int(ped['status']),
        pedDir,
        pedSpeed,
        int(pas['status']),
        pasDestX/60.0, # these should be normalized to 1 for NN
        pasDestY/60.0, # these should be normalized to 1 for NN
        pasIsInMyCar
      ] 
      
      self.state.append(np.array(stateOfPoint))
      
    self.state=np.array(self.state).flatten()
    self.state=np.append(self.state, np.array([
      self.myCar['pos']['x']/60.0, # these should be normalized to 1 for NN
      self.myCar['pos']['y']/60.0, # these should be normalized to 1 for NN
      self.directionCodes[self.myCar['direction']],
      self.myCar['speed'],
      self.myCar['life']/100.0 # these should be normalized to 1 for NN
    ]))
    return self.state

In [7]:
"""
  Baseline rewarder: go to a certain point, without dying.
"""
class RewarderBaseline:
  pass

In [8]:
"""
  Baseline state-space: go to a certain point, without dying.
"""
class StateSpaceBaseline:
  pass

In [31]:
## Test section
g = CCGame(log_ticks=True)
g.start()
g.send_random_command()
g.send_random_command()
g.close()

print_t("Game id: {}".format(g.game_id))
print_t("Tick data: \n{}".format(g.tick_data))
print_t("Tick log ({0} entries): \n{1}".format(len(g.tickLogs),g.tickLogs))

[2019-04-13 21:04:01] Game id: 16443
[2019-04-13 21:04:01] Tick data: 
{'passengers': [{'dest_pos': {'x': 58, 'y': 12}, 'id': 16, 'pos': {'x': 15, 'y': 25}}, {'dest_pos': {'x': 16, 'y': 48}, 'id': 17, 'pos': {'x': 19, 'y': 21}}, {'dest_pos': {'x': 47, 'y': 23}, 'id': 18, 'pos': {'x': 34, 'y': 6}}, {'dest_pos': {'x': 58, 'y': 12}, 'id': 19, 'pos': {'x': 45, 'y': 55}}, {'dest_pos': {'x': 12, 'y': 6}, 'id': 20, 'pos': {'x': 34, 'y': 58}}], 'pedestrians': [], 'cars': [{'life': 97, 'transported': 0, 'speed': 0, 'direction': 'DOWN', 'command': '0', 'id': 0, 'pos': {'x': 8, 'y': 2}}, {'life': 100, 'transported': 0, 'speed': 0, 'direction': 'UP', 'command': '0', 'id': 21, 'pos': {'x': 8, 'y': 3}}], 'request_id': {'tick': 2, 'car_id': 0, 'game_id': 16443}, 'messages': []}
[2019-04-13 21:04:01] Tick log (3 entries): 
[{'passengers': [{'dest_pos': {'x': 58, 'y': 12}, 'id': 16, 'pos': {'x': 15, 'y': 25}}, {'dest_pos': {'x': 16, 'y': 48}, 'id': 17, 'pos': {'x': 19, 'y': 21}}, {'dest_pos': {'x': 47,

In [28]:
g.close()
"g.started = {}".format(g.started)

[2019-04-13 21:02:29] Game already closed.


'g.started = False'

In [ ]:
len(g.)

In [32]:
ss = StateSpace()
state = ss.stateFromTick(g)

[2019-04-13 21:14:30] My car is {'life': 97, 'transported': 0, 'speed': 0, 'direction': 'DOWN', 'command': '0', 'id': 0, 'pos': {'x': 8, 'y': 2}}


In [91]:
state

array([ 0.        ,  0.        ,  0.        , -1.        ,  0.        ,
        0.        , -1.        ,  0.        , -0.01666667, -0.01666667,
       -1.        ,  0.        ,  0.        ,  0.        , -1.        ,
        0.        ,  0.        , -1.        ,  0.        , -0.01666667,
       -0.01666667, -1.        ,  2.        ,  0.        ,  0.        ,
       -1.        ,  0.        ,  0.        , -1.        ,  0.        ,
       -0.01666667, -0.01666667, -1.        ,  0.        ,  0.        ,
        0.        , -1.        ,  0.        ,  0.        , -1.        ,
        0.        , -0.01666667, -0.01666667, -1.        ,  0.        ,
        0.        ,  0.        , -1.        ,  0.        ,  0.        ,
       -1.        ,  0.        , -0.01666667, -0.01666667, -1.        ,
        0.        ,  0.        ,  0.        , -1.        ,  0.        ,
        0.        , -1.        ,  0.        , -0.01666667, -0.01666667,
       -1.        ,  0.        ,  0.        ,  0.        , -1.  

In [83]:
g.close()

[2019-04-09 16:30:36] Game already closed.


In [84]:
rewarder = Rewarder()
def testGameTime():
  g = CCGame()
  g.start()
  while not g.crashed:
    g.send_random_command()
    print_t("Reward: {}".format(rewarder.calculate_reward(g)))
  print_t("ticks: {}".format(g.ticknum))
  g.close()
  return

In [85]:
testGameTime()

[2019-04-09 16:30:40] Reward: 0
[2019-04-09 16:30:40] Reward: 0
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -2
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: 0
[2019-04-09 16:30:41] Reward: 0
[2019-04-09 16:30:41] Reward: 0
[2019-04-09 16:30:41] Reward: 0
[2019-04-09 16:30:41] Reward: -2
[2019-04-09 16:30:41] Reward: -4
[2019-04-09 16:30:41] Reward: -4
[2019-04-09 16:30:41] Reward: -3
[2019-04-09 16:30:41] Reward: -2
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -2
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: 0
[2019-04-09 16:30:41] Reward: -5
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30:41] Reward: -4
[2019-04-09 16:30:41] Reward: -2
[2019-04-09 16:30:41] Reward: -4
[2019-04-09 16:30:41] Reward: -1
[2019-04-09 16:30

In [92]:
%%timeit
testGameTime()

[2019-04-09 17:36:29] Reward: 0
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: -3
[2019-04-09 17:36:30] Reward: -1
[2019-04-09 17:36:30] Reward: -76
[2019-04-09 17:36:30] Reward: -17
[2019-04-09 17:36:30] CAR CRASHED.
[2019-04-09 17:36:30] Reward: -3
[2019-04-09 17:36:30] ticks: 7
[2019-04-09 17:36:30] Game already closed.
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: -1
[2019-04-09 17:36:30] Reward: -2
[2019-04-09 17:36:30] Reward: -2
[2019-04-09 17:36:30] Reward: -2
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: -3
[2019-04-09 17:36:30] Reward: -3
[2019-04-09 17:36:30] Reward: -2
[2019-04-09 17:36:30] Reward: -1
[2019-04-09 17:36:30] Reward: -1
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: -2
[2019-04-09 17:36:30] Reward: -2
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: 0
[2019-04-09 17:36:30] Reward: -1
[2019-04

[2019-04-09 17:36:34] Reward: 0
[2019-04-09 17:36:34] Reward: -3
[2019-04-09 17:36:34] Reward: -1
[2019-04-09 17:36:34] Reward: -3
[2019-04-09 17:36:34] Reward: -4
[2019-04-09 17:36:34] Reward: -1
[2019-04-09 17:36:34] Reward: -2
[2019-04-09 17:36:34] Reward: -3
[2019-04-09 17:36:34] Reward: -2
[2019-04-09 17:36:34] Reward: -1
[2019-04-09 17:36:34] Reward: -1
[2019-04-09 17:36:34] Reward: -1
[2019-04-09 17:36:34] Reward: -4
[2019-04-09 17:36:34] Reward: -5
[2019-04-09 17:36:35] Reward: -3
[2019-04-09 17:36:35] Reward: -3
[2019-04-09 17:36:35] Reward: -2
[2019-04-09 17:36:35] Reward: -2
[2019-04-09 17:36:35] Reward: -1
[2019-04-09 17:36:35] Reward: -1
[2019-04-09 17:36:35] Reward: 0
[2019-04-09 17:36:35] Reward: 0
[2019-04-09 17:36:35] Reward: 0
[2019-04-09 17:36:35] Reward: -3
[2019-04-09 17:36:35] Reward: -1
[2019-04-09 17:36:35] Reward: -4
[2019-04-09 17:36:35] Reward: -5
[2019-04-09 17:36:35] Reward: -1
[2019-04-09 17:36:35] Reward: -4
[2019-04-09 17:36:35] Reward: -2
[2019-04-09 17

In [38]:
""" TODO: As a baseline project, go to a certain point without dying. Use networkx
to find the shortest path to the point.
"""

' TODO: As a baseline project, go to a certain point without dying. Use networkx\nto find the shortest path to the point.\n'

In [39]:
a=[{"id": 2, "life": 100}, {"id": 0, "life":80}]
list(filter(lambda x: x["id"]==0, a))

[{'id': 0, 'life': 80}]

In [90]:
""" 
   Deep Q Network 
"""
import random
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

class DQNAgent:
  def __init__(self):
    self.gamma = 0.9
    self.memory = []
    self.learning_rate = 0.0005
    self.network_input_shape = (1,)
    self.actions = np.array(["NO_OP", "ACCELERATION", "DECELERATION", "CAR_INDEX_LEFT", "CAR_INDEX_RIGHT", "CLEAR", "FULL_THROTTLE", "EMERGENCY_BRAKE", "GO_LEFT", "GO_RIGHT"])
    self.network = None
    return
  
  def baseline_network(self, weights=None):
    # This returns a tensor
    inputs = Input(shape=self.network_input_shape)

    # a layer instance is callable on a tensor, and returns a tensor
    x = Dense(output_dim=120, activation='relu')(inputs)
    x = Dropout(0.15)(x)
    x = Dense(output_dim=120, activation='relu')(x)
    x = Dropout(0.15)(x)
    x = Dense(output_dim=120, activation='relu')(x)
    x = Dropout(0.15)(x)
    predictions = Dense(len(self.actions), activation='softmax')(x)

    model = Model(inputs=inputs, outputs=predictions)
    opt = Adam(self.learning_rate)
    model.compile(loss='mse', optimizer=opt)
    
    if weights:
      model.load_weights(weights)
    return model
  
  def load_network_hdf5(self):
    pass # Implement loading a hdf5 network 
  
  def save_network_hdf5(self, filename):
    pass # Implement saveing a network to hdf5
  
  def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))
    
  
  pass

In [41]:
jsonstr = '[{"x":-1,"y":-7},{"x":0,"y":-7},{"x":1,"y":-7},{"x":-1,"y":-6},{"x":0,"y":-6},{"x":1,"y":-6},{"x":-1,"y":-5},{"x":0,"y":-5},{"x":1,"y":-5},{"x":-1,"y":-4},{"x":0,"y":-4},{"x":1,"y":-4},{"x":-2,"y":-3},{"x":-1,"y":-3},{"x":0,"y":-3},{"x":1,"y":-3},{"x":2,"y":-3},{"x":-3,"y":-2},{"x":-2,"y":-2},{"x":-1,"y":-2},{"x":0,"y":-2},{"x":1,"y":-2},{"x":2,"y":-2},{"x":3,"y":-2},{"x":-3,"y":-1},{"x":-2,"y":-1},{"x":-1,"y":-1},{"x":0,"y":-1},{"x":1,"y":-1},{"x":2,"y":-1},{"x":3,"y":-1},{"x":-3,"y":0},{"x":-2,"y":0},{"x":-1,"y":0},{"x":1,"y":0},{"x":2,"y":0},{"x":3,"y":0},{"x":-2,"y":1},{"x":-1,"y":1},{"x":0,"y":1},{"x":1,"y":1},{"x":2,"y":1},{"x":-1,"y":2},{"x":0,"y":2},{"x":1,"y":2},{"x":0,"y":3}]'

In [42]:
data = json.loads(jsonstr)

In [43]:
[data[k]['x'] == ss.viewAreaCoords[k][0] and data[k]['y']==ss.viewAreaCoords[k][1] for k in range(len(ss.viewAreaCoords))]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [44]:
np.round([0.991, 0.0001]).astype(int)

array([1, 0])

In [45]:
#x = {"status": False, "car": None}
x = {"status": True, "car": {"direction": "UP"}}

0 if x['status']==False else x['car']['direction']

'UP'

In [46]:
o={"a": {"b": 2}}

In [47]:
o["a"]["b"]

2

In [48]:
a = {"p": 1, "q": 3}
b = a 
a = {"p":2, "q": 2}

In [49]:
b

{'p': 1, 'q': 3}

In [52]:
a = ['END REASON: CRASHED', 'jsadkasjdkl']
'crash' if 'END REASON: CRASHED' in a else 'no crash'

'crash'

In [16]:
a = {"alma": [], "korte": {}}
if ("szar" in a.keys() and len(a['szar']) > 0):
  print("van szar")